In [1]:
# cut in x-direction
sagittal = -25
# cut in y-direction
coronal = -37
# cut in z-direction
axial = -6

# coordinates displaying should be prepared as a list
cut_coords = [sagittal, coronal, axial]

In [2]:
# We load data from nilearn by import datasets
from nilearn import datasets

# First, we fetch single subject specific data with haxby datasets: to have
# anatomical image, EPI images and masks images
haxby_dataset = datasets.fetch_haxby()

# print basic information on the dataset
print('First subject anatomical nifti image (3D) located is at: %s' %
      haxby_dataset.anat[0])
print('First subject functional nifti image (4D) is located at: %s' %
      haxby_dataset.func[0])
print('Labels of haxby dataset (text file) is located at: %s' %
      haxby_dataset.session_target[0])

# Second, load the labels stored in a text file into array using pandas
import pandas as pd

session_target = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
# Now, we have the labels and will be useful while computing student's t-test
haxby_labels = session_target['labels']

/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First subject anatomical nifti image (3D) located is at: /Users/nikkibytes/nilearn_data/haxby2001/subj2/anat.nii.gz
First subject functional nifti image (4D) is located at: /Users/nikkibytes/nilearn_data/haxby2001/subj2/bold.nii.gz
Labels of haxby dataset (text file) is located at: /Users/nikkibytes/nilearn_data/haxby2001/subj2/labels.txt


In [3]:
# Smooth the data using image processing module from nilearn
from nilearn import image

# Functional data
fmri_filename = haxby_dataset.func[0]
# smoothing: first argument as functional data filename and smoothing value
# (integer) in second argument. Output returns in Nifti image.
fmri_img = image.smooth_img(fmri_filename, fwhm=6)

# Visualize the mean of the smoothed EPI image using plotting function
# `plot_epi`
from nilearn.plotting import plot_epi

# First, compute the voxel-wise mean of smooth EPI image (first argument) using
# image processing module `image`
mean_img = image.mean_img(fmri_img)
# Second, we visualize the mean image with coordinates positioned manually
plot_epi(mean_img, title='Smoothed mean EPI', cut_coords=cut_coords)

In [8]:
fmri_data = fmri_img.get_data()
# number of voxels being x*y*z, samples in 4th dimension
print(fmri_data.shape)


(40, 64, 64, 1452)


In [5]:
import numpy as np
from scipy import stats

# This test returns p-values that represent probabilities that the two
# time-series were not drawn from the same distribution. The lower the
# p-value, the more discriminative is the voxel in distinguishing the two
# conditions (faces and houses).
_, p_values = stats.ttest_ind(fmri_data[..., haxby_labels == 'face'],
                              fmri_data[..., haxby_labels == 'house'],
                              axis=-1)


In [12]:
fmri_data

array([[[[3.82783949e-01, 3.15722167e-01, 1.93227097e-01, ...,
          8.88536274e-01, 1.79597533e+00, 9.63508844e-01],
         [4.48387653e-01, 3.57153237e-01, 2.62456715e-01, ...,
          3.60178852e+00, 6.74118948e+00, 3.68342948e+00],
         [3.93492043e-01, 3.94156963e-01, 3.09761494e-01, ...,
          5.81759548e+00, 9.12431622e+00, 5.42240191e+00],
         ...,
         [5.35980606e+00, 3.80976176e+00, 4.74490786e+00, ...,
          3.10985953e-01, 3.99323791e-01, 4.01381314e-01],
         [2.98107243e+00, 2.08783507e+00, 1.77438378e+00, ...,
          1.66237265e-01, 2.97099471e-01, 2.51354843e-01],
         [6.65085733e-01, 4.57098573e-01, 2.93900251e-01, ...,
          3.76221165e-02, 1.17603317e-01, 6.04814105e-02]],

        [[1.31131351e+00, 1.08614349e+00, 7.85015523e-01, ...,
          7.95306444e-01, 1.51361012e+00, 1.36506402e+00],
         [1.50369680e+00, 1.22958148e+00, 1.05018675e+00, ...,
          3.34878469e+00, 5.61668968e+00, 4.88832569e+00],
        